In [1]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
from datetime import datetime
workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

def get_folders_in_dir(dir):
    return [f for f in os.listdir(dir) if os.path.isdir(os.path.join(dir, f))]

trash = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/band_intermediate/'
temp = '/data/Aldhani/eoagritwin/et/Auxiliary/trash/vrt/'
os.makedirs(trash, exist_ok=True)
os.makedirs(temp, exist_ok=True)

In [2]:
# read csv for valid row_cols for samples to draw. They are based on the share of agriculture (HR Landcover maps) within a S3 pixel
thresh_csv = pd.read_csv(f'/data/{origin}et/Auxiliary/landcover/csv/row_cols.csv')

for col in thresh_csv.columns:
    #print(f'finding indices for {col}')
    nested = [entry.split('_') for entry in thresh_csv[col] if type(entry) == str]
    rows, cols = zip(*nested)
    #print(len(rows))
# only use Thresh50, as all valid row_col will be here. For other THresh sets, a subset from this can be taken
# this are the last instances of rows and cols
rows = [int(row) for row in rows]
cols = [int(col) for col in cols]

/tmp/ipykernel_3395698/3154273301.py:2: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  thresh_csv = pd.read_csv(f'/data/{origin}et/Auxiliary/landcover/csv/row_cols.csv')


In [2]:
# check if all data is there and get the the different stacks (indices)
year = 2019
force_path = f'/data/Aldhani/eoagritwin/force/output/S3/{year}/'
folders = get_folders_in_dir(force_path)
folder_to_clean = [os.path.join(force_path, folder) for folder in folders if len(getFilelist(os.path.join(force_path, folder), '.tif', deep=True)) != 6]
if len(folder_to_clean) > 0:
    raise ValueError('the folders do not contain the same number of images...')
# for folder in folder_to_clean:
#     [os.remove(file) for file in getFilelist(folder, '.tif', deep=True) if '_EVI_' in file]
indices = list(set([file_annual.split('_')[-2] for file_annual in getFilelist(os.path.join(force_path, folders[0]), '.tif', deep=True)]))
indices.sort()


ValueError: the folders do not contain the same number of images...

In [3]:
folder_to_clean

['/data/Aldhani/eoagritwin/force/output/S3/2019/X0073_Y0049']

In [9]:
# find the bands in raster stacks that belong to the year of interest
ds = gdal.Open(getFilelist(os.path.join(force_path, folders[0]), '.tif', deep=True)[0], 0)
if ds.RasterCount > 366:
    bandname = str(ds.GetRasterBand(93).GetDescription())
    if bandname == f'{year}0101':
        start_index = 93
        end_index = 457
        if year in [2016,2020,2024]:
            end_index +=1
    else:
        raise ValueError('band for Jan 01 not found!')

files = [getFilelist(os.path.join(force_path, folder), '.tif', deep=True) for folder in folders]
files = [file for list in files for file in list]

/home/potzschf/mambaforge/envs/workhorse/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [10]:
#for index in indices:
index = indices[1]
index_files = [file for file in files if index in file]
conti = []
for band_number in range(start_index,end_index+1,1):
    # create daily vrts and extract

    for i, tif in enumerate(index_files):
        output = trash + tif.split('/')[-1].split('.')[0] + '_' + str(band_number) + '_' + str(i) + '.tif'
        gdal.Translate(output, tif, bandList=[band_number])

    # Now build VRT
    vrt_options = gdal.BuildVRTOptions(separate=False)
    vrt_ds = gdal.BuildVRT(f'{temp}{year}_{index}_{band_number}.vrt', getFilelist(trash, '.tif'), options=vrt_options)
    # vrt_ds.FlushCache()
    # vrt = gdal.Open(f'{temp}{year}_{index}_{band_number}.vrt')
    warpi = warp_to_template(vrt_ds, 
            '/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif',
            mask_path='/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif',
            #outPath=f'/data/Aldhani/eoagritwin/et/FORCE/vrt_dumps/{year}_{index}_{band_number}.tif',
            outType=gdal.GDT_Int16)
    conti.append(warpi)
    [os.remove(file) for file in getFilelist(trash, '.tif')]

KeyboardInterrupt: 

In [ ]:
# sanity check
arr = warpi[rows,cols]
print(f'the shape after fancy indexing:{arr.shape}\ncheck if the extracted values are stored row-wise in new 1d array')
a = 5
print(f'at a randomly selected index for a row({a}) and one for a col({a})\nthis related to the agri-indices at row:{rows[a]} and col:{cols[a]}')

print(f'\nTherefore warpi[rows[a], cols[b]] results in {warpi[rows[a], cols[a]]}')

rl = [e for e, row in enumerate(rows) if row == rows[a]]
cl = [e for e, col in enumerate(cols) if col == cols[a]]
indi = [r for r in rl if r in cl]

print(f'This should be the same as {arr[indi][0]}')


the shape after fancy indexing:(165877,)
check if the extracted values are stored row-wise in new 1d array
at a randomly selected index for a row(5) and one for a col(8079)
this related to the agri-indices at row:100 and col:367

Therefore warpi[rows[a], cols[b]] results in 1707
This should be the same as 1707


In [31]:
for a in range(arr.shape[0]):
    r1 = warpi[rows[a], cols[a]]
    rl = [e for e, row in enumerate(rows) if row == rows[a]]
    cl = [e for e, col in enumerate(cols) if col == cols[a]]
    indi = [r for r in rl if r in cl]
    r2 = arr[indi][0]
    if r1 != r2:
        print('noooooo')
